<h2>Company Sentiment Analysis</h2>
<p>This script analyses the predictions made by a classifications algorithm to determine the best company for investing in.</p>
<p>A score is calculated for each company based on the percentage of positive, neutral and negative predictions for each application within a company to allow the companies to be compared</p>
<p><b>Score Calculation:</b> The score for each company is calculated by:</p>
<ol>
    <li>Averaging the number of predictions for each of the sentiment classes</li>
    <li>Using neutral reviews as the base for the calculations <br>$score = neutral$</li>
    <li>The averages for positive and negative reviews are doubled <br>$positive = positive * 2$<br> $negative = negative * 2$</li>
    <li>Positive average being added to the score <br>$score = score + positive$</li>
    <li>Negative average being subtracted from the result of the previous calculation <br>$score = score - negative$</li>
    <li>Dividing the score by 2 <br>$score = \frac{score}{2}$</li>

$$score = \frac{neutral + (positive * 2) - (negative * 2)}{2}$$

<h3>Import Modules</h3>
<p>This block imports the pandas package to process the CSV file containing the predictions and product ID's, the OS package to access the file system and fnmatch to identify .csv files within a directory</p>

In [ ]:
import pandas as pd
import os, fnmatch

<h3>load_sentiment_data (input_csv)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>input_csv</b> - [string] filename of the file containing the Amazon reviews</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>product_ids</b> - [list] Product ID the review relates to</li>
            <li><b>predicted_sentiment</b> - [list] Sentiment class of the review (as predicted by the algorithm)</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>load_sentiment_data()</i> method is used to extract the product ID and predicted sentiment class of the review from a file and convert them into a DataFrame</li>
    <li>Once this is complete, the two columns of the DataFrame are returned</li>
   </ol>

In [ ]:
def load_sentiment_data(input_csv):
    df = pd.read_csv(input_csv,
                    delimiter=',',
                    header=0)
    product_ids = df['product_id']
    predicted_sentiment = df['predicted_sentiment_score']
    return product_ids, predicted_sentiment

<h3>get_numeric_user_input (prompt, retry_limit, minimum, maximum)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>prompt</b> - [string] Message that will be displayed to the user</li>
            <li><b>retry_limit</b> - [integer] Number of times a user can retry after an incorrect input</li>
            <li><b>minimum</b> - [integer] Number that user input cannot be below</li>
            <li><b>maximum</b> - [integer] Number that user input cannot be above</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>user_input</b> - [integer] The number the user entered</li>
            <li>OR</li>
            <li><b>NONE</b> - </li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>get_numeric_user_input()</i> method is used to get a number input from the user by firstly checking is the user has run out of retries (<i>retry_limit</i>)</li>
    <li>Once this is complete, the user is prompted (with the contents of the <i>prompt</i> parameter) to enter a number, which is attempted to be converted to an integer within the <i>try</i> block</li>
    <li>The number is then evaluated to check it is within the <i>minimum</i> and <i>maximum</i> parameters
        <ul>
            <li>If it does not meet these requirements, an exception is raised</li>
        </ul>
    </li>
    <li>The <i>user_input</i> is returned</li>
    <li><b>if an exception is raised: </b> (either because the input is not a number or is not within the <i>minimum</i> and <i>maximum</i> parameters
        <ul>
            <li>The <i>retry_limit</i> is decreased by one (as the user has had an attempt)</li>
            <li>An error message is printed to the user, and while the <i>retry_limit</i> is greater than zero, the user can retry</li>
            <li>If the retry limit reaches zero, the user will not be prompted for input and an unhandled exception will be raised, meaning that users cannot continually enter incorrect values beyond the <i>retry_limit</i> and the script will stop executing</li>
        </ul>
    </li>
</ol>

In [ ]:
def get_numeric_user_input(prompt, retry_limit, minimum, maximum):
    while retry_limit > 0:
        print("")
        try:
            user_input = int(input(prompt))
            if user_input < minimum or user_input > maximum:
                raise
            return user_input
        except:
            retry_limit -= 1
            print("*** Please enter a number between", minimum, "&", maximum, "***")
            print("You have", retry_limit, "attempts remaining")
    return

<h3>choose_predictions_file ()</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>NONE</b></li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>path</b> - [string] Path to file location</li>
            <li><b>file</b> - [string] Filename</li>
            <li>OR</li>
            <li><b>NONE</b></li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>choose_predictions_file()</i> method is used to allow the user to choose a file containing predictions from those output in the algorithm analysis</li>
    <li>The method defines the path, a list of files in the path (<i>dir_list</i>) and stores a list of CSV files contained in the file path location
        <ul>
            <li>HOFMANN, F., 2017. Python: List Files in a Directory. <i>Stack Abuse</i> [online]. Available from: https://stackabuse.com/python-list-files-in-a-directory/ [Accessed 21 Dec 2018].

</li>
        </ul>
    </li>
    <li>The stored filenames are printed with numbers for the user to make a selection</li>
    <li>The <i>get_numeric_user_input()</i> method is called to allow the user to make a file selection</li>
    <li>The method then identifies the corresponding file from the number the user entered and returns the filename and path</li>
</ol>

In [ ]:
def choose_predictions_file():
    path = 'Predictions/'
    dir_list = os.listdir(path)
    file_list = []
    counter = 1
    for entry in dir_list:  
        if fnmatch.fnmatch(entry, "*.csv"):
            file_list.append([entry, counter])
            counter += 1
    for file, number in file_list:
        print("[" + str(number) + "]", file)
    user_file = get_numeric_user_input("Enter the number of the file you would like to use: ", 3, 1, len(file_list))
    for file, number in file_list:
        if user_file == number:
            return path, file

<h3>choose_predictions_file ()</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>NONE</b></li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Contains the company &amp; application names along with metrics for the applications and companies</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>create_data_structure()</i> method is used to create the data dictionary structure which will be used to hold all of the metrics about the applications and companies</li>
    <li>The company and application names are defines and stored in the <i>company_applications</i> dictionary</li>
    <li>The companies are then iterated over to add average percentage counts (as these are the same initially for each company)</li>
    <li>The applications are iterated over and the sentiment cound, percentages and total fields are added for each application</li>
    <li>The dictionary <i>company_applications</i> is then returned</li>
</ol>

In [ ]:
def create_data_structure():
    company_applications = {'AAD_1': {'apps': {'B004NWLM8K': {}, 'B004Q1NH4U': {}, 'B004LPBTAA': {}}},
                            'AAD_2': {'apps': {'B004S6NAOU': {}, 'B004R6HTWU': {}, 'B004N8KDNY': {}}},
                            'AAD_3': {'apps': {'B004KA0RBS': {}, 'B004NPELDA': {}, 'B004L26XXQ': {}}}}
    for company in company_applications:
        company_applications[company]['avg_percentages'] = {1:0, 2:0, 3:0}
        for application in company_applications[company]['apps']:
            company_applications[company]['apps'][application]['sent_counts'] = {1:0, 2:0, 3:0}
            company_applications[company]['apps'][application]['percentages'] = {1:0, 2:0, 3:0}
            company_applications[company]['apps'][application]['total'] = 0  
    return company_applications

<h3>sentiment_counter_per_aplication (company_applications)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications (updated application sentiment counts)</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>sentiment_counter_per_application()</i> method is used to count the number of occurences of each sentiment class for each application</li>
    <li>The length of the <i>product_ids</i> list is checked against the length of the <i>predicted_sentiment</i> list to ensure they match</li>
    <li>The <i>product_ids</i> list is iterated over to identify if the product ID matches an application in the dictionary</li>
    <li>If a match is found, the <i>sent_counts</i> attribute is incremented for the specific sentiment class</li>
    <li>The modified <i>company_applications</i> dictionary is returned with the sentiment counts for each application</li>
</ol>

In [ ]:
def sentiment_counter_per_application(company_applications):
    if len(product_ids == len(predicted_sentiment)):
        for i in range(len(product_ids)):
            for company in company_applications:
                for application in company_applications[company]['apps']:
                    if product_ids[i] == application:
                        company_applications[company]['apps'][application]['sent_counts'][predicted_sentiment[i]] += 1
    return company_applications

<h3>sentiment_totals_per_application (company_applications)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications (updated application sentiment totals)</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>sentiment_totals_per_application()</i> method is used to gather the total number of sentiment counts for each application across all classes</li>
    <li>The companies are iterated over and the applications</li>
    <li>The sentiment count for each class is added together and stored as a total for the application <i>[application]['total']</i></li>
    <li>The modified <i>company_applications</i> dictionary is then returned</li>
</ol>

In [ ]:
def sentiment_totals_per_application(company_applications):
    for company in company_applications:
        for application in company_applications[company]['apps']:
            for sentiment in company_applications[company]['apps'][application]['sent_counts']:
                company_applications[company]['apps'][application]['total'] += company_applications[company]['apps'][application]['sent_counts'][sentiment]
    return company_applications

<h3>calculate_sentiment_percentage (company_applications)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications (updated application sentiment percentages)</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>calculate_sentiment_percentage()</i> method is used to calculate the percentage for each application sentiment count against for each class per application</li>
    <li>The companies are iterated over, along with the applications and their sentiment counts</li>
    <li>The sentiment count for each class is divided by the total number of sentiment values for the application, multiplied by 100 to create a percentage</li>
    <li>The percentage is rounded to two decimal places as it is not necessary to compare beyond that</li>
    <li>The modified <i>company_applications</i> dictionary is then returned</li>
</ol>

In [ ]:
def calculate_sentiment_percentage(company_applications):
    for company in company_applications:
        for application in company_applications[company]['apps']:
            for sentiment in company_applications[company]['apps'][application]['sent_counts']:
                percentage = round((company_applications[company]['apps'][application]['sent_counts'][sentiment] / company_applications[company]['apps'][application]['total']) * 100, 2)
                company_applications[company]['apps'][application]['percentages'][sentiment] = percentage
    return company_applications

<h3>calculate_average_sentiment_percentage_for_company (company_applications)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications (updated company average sentiment percentages)</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>calculate_average_sentiment_percentage_for_company()</i> method is used to calculate the average percentage for each sentiment class of the company's applications</li>
    <li>The companies are iterated over, along with the applications and their sentiment percentages (per class)</li>
    <li>The percentage is added to the <i>[avg_percentages]</i> for each of the class and the total is divided by the amount of applications a company has</li>
    <li>This generates a percentage value for each sentiment class for each company</li>
    <li>The modified <i>company_applications</i> dictionary is then returned</li>
</ol>

In [ ]:
def calculate_average_sentiment_percentage_for_company(company_applications):
    for company in company_applications:
        for application in company_applications[company]['apps']:
            for percentage in company_applications[company]['apps'][application]['percentages']:
                company_applications[company]['avg_percentages'][percentage] += company_applications[company]['apps'][application]['percentages'][percentage]
        for i in range(len(company_applications[company]['apps'])):
            company_applications[company]['avg_percentages'][i + 1] = round(company_applications[company]['avg_percentages'][i + 1] / len(company_applications[company]['apps']), 2)
    return company_applications

<h3>sort_list (list)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>list</b> - [list] A list of items to be sorted</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>list</b> - [list] The sorted list of items</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>sort_list()</i> method is used to sort the values in a list in descending order</li>
    <li>The <i>sort()</i> method is called on the list with the <i>reverse</i> parameter set to true (high -> low)</li>
    <li>The sorted list is returned</li>
</ol>

In [ ]:
def sort_list(list):
    list.sort(reverse=True)
    return list

<h3>get_average_percentage_for_sentiment (sentiment, company_applications)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>sentiment</b> - [integer] Sentiment class to be retrieved</li>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>avg_percentages</b> - [list] a sorted list of average percentages for the specified sentiment class</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>get_average_percentage_for_sentiment()</i> method is used to retrieve the average sentiment percentage for a specified sentiment class for each company</li>
    <li>the percentage and the company it belongs to are appended to the <i>avg_percentages</i> list</li>
    <li>The <i>sort_list()</i> method is called with the <i>avg_percentages</i> list, and is then returned once the list has been sorted</li>
</ol>

In [ ]:
def get_average_percentage_for_sentiment(sentiment, company_applications):
    avg_percentages = []
    for company in company_applications:
        avg_percentages.append([company_applications[company]['avg_percentages'][sentiment], company])
    return sort_list(avg_percentages)

<h3>get_average_percentages_for_company (company, company_applications)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>company</b> - [string] Name of the company</li>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>avg_list</b> - [list] a list of average percentages for the specified company</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>get_average_percentages_for_company()</i> method is used to retrieve the average percentages for a company among all sentiment classes</li>
    <li>The sentiment classes are iterated over and the percentage is appended to the <i>avg_list</i> list</li>
    <li>The <i>avg_list</i> is then returned</li>
</ol>

In [ ]:
def get_average_percentages_for_company(company, company_applications):
    avg_list = []
    for i in company_applications[company]['avg_percentages']:
        avg_list.append(company_applications[company]['avg_percentages'][i])
    return avg_list

<h3>calculate_company_score (averages)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>averages</b> - [list] Name of the company</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>score</b> - [float] The score for the company</li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>calculate_company_score()</i> method is used to calculate a score for a company</li>
    <li>The score is assigned the neutral value <i>averages[1]</i></li>
    <li>The positive average <i>averages[2]</i> is then doubled and added to the score</li>
    <li>The negative average <i>averages[0]</i> is then doubled and subtracted from the score</li>
    <li>The <i>score</i> is then divided by two, to make the score out of 100, rather than 200 due to the doubled positive and negative averages</li> 
    <li>The <i>score</i> is then returned</li>
</ol>

In [ ]:
def calculate_company_score(averages):
    score = averages[1] + (averages[2] * 2) - (averages[0] * 2)
    score = round(score / 2, 2)
    return score

<h3>calculate_best_company (company_applications)</h3>
<ul>
    <li><u>Inputs</u>
        <ul>
            <li><b>company_applications</b> - [dictionary] Used to store all data about companies and applications</li>
        </ul>
    </li>
    <li><u>Outputs</u>
        <ul>
            <li><b>NONE</b></li>
        </ul>
    </li>
</ul>
<h4>What does this method do?</h4>
<ol>
    <li>The <i>calculate_best_company()</i> method is identify the best company</li>
    <li>The best company is the one with the highest score, as positive reviews increase the score and negative reviews decrease the score</li>
    <li>The average percentages for each company are retrieved and stored in <i>avg_percentages</i> using the <i>get_average_percentages_for_company()</i> method</li>
    <li>The company scores are calculated using the <i>avg_percentages</i> data as a parameter for the <i>calculate_company_score()</i> method</li>
    <li>The score is added to the <i>scores</i> list, which is then sorted</li>
    <li>The scores are printed for the user to see in descending order</li>
    <li>The best score is highlighted and the difference between the companies scores are provided</li>
    <li>The number of predictions for each sentiment score per company is displayed</li>
</ol>

In [ ]:
def calculate_best_company(company_applications):
    scores = []
    for company in company_applications:
        avg_percentages = get_average_percentages_for_company(company, company_applications)
        company_score = calculate_company_score(avg_percentages)
        scores.append([company_score, company])
        
    scores = sort_list(scores)
    print("From the data contained in " + filename + ", the following scores have been calculated:")
    print("(The higher the score, the better)\n")
    for score, company in scores:
        print(str(company) + ': ', score)
    print("")
    print("The company with the best score is:", scores[0][1], "\n")
    print("Difference between company with the best score and the second best score:", scores[0][0] - scores[1][0])
    print("\n\n")
    
    for company in company_applications:
        sent_1 = 0
        sent_2 = 0
        sent_3 = 0
        for application in company_applications[company]['apps']:
            sent_1 += company_applications[company]['apps'][application]['sent_counts'][1]
            sent_2 += company_applications[company]['apps'][application]['sent_counts'][2]
            sent_3 += company_applications[company]['apps'][application]['sent_counts'][3]
        print(company, "Sentiment Breakdown")
        print("-------------------------")
        print("Negative    ", sent_1)
        print("Neutral     ", sent_2)
        print("Positive    ", sent_3, "\n")
        print("Total       ", sent_1 + sent_2 + sent_3)
        print("-------------------------\n\n")
    
#     print(company_applications)

<h3>Run the Analysis</h3>
<p>This block calls all of the methods in turn (and from inside functions) to analyse the companies scores. The data structure is created and a file is selected by the user, then the sentiment values are totalled and averaged as well as company averages to generate the results</p> 

In [ ]:
filepath, filename = choose_predictions_file()
product_ids, predicted_sentiment = load_sentiment_data(filepath+filename)

print()
print("*** Just doing some calculations, please wait ***\n")

company_applications = create_data_structure()
company_applications = sentiment_counter_per_application(company_applications)
company_applications = sentiment_totals_per_application(company_applications)
company_applications = calculate_sentiment_percentage(company_applications)
company_applications = calculate_average_sentiment_percentage_for_company(company_applications)

print("Calculations complete, results are shown below\n\n")

print("-------------------------------------------------------------------------------\n")
print("RESULTS\n")

calculate_best_company(company_applications)